# Results and Insights
Now that we've gauged the performance of the three fp-growth algorithms, we can do some thorough analysis on the ages dataset. Let's first get set up.

In [86]:
import pandas as pd
import matplotlib.pyplot as plt

# custom fp-tree implementation
from fptree import Tree

# utility function to print frequent itemsets from dictionary representation
# both pyfpgrowth and custom implementation return the frequent itemsets in this form
def print_freq_itemsets(freq_itemsets, num_transactions):    
    for itemset, sup_count in freq_itemsets.items():
        str = "{"
        for item in itemset:
            str += f"{item}, "
        str = str[:-2]
        str += "}"
        print(f'{str} | sup_count: {sup_count} | support: {sup_count/num_transactions:.3f}')

In [87]:
trans_df = pd.read_csv("ages_transaction.csv")
trans_df = trans_df[['Gender', 'Occupation', 'Manner of death', 'AOD_Category', 'Associated Countries']]
trans_df.head(10)

Gender               Occupation              Manner of death   
0  GENDER_male        OCCUPATION_artist           MOD_natural causes  \
1  GENDER_male    OCCUPATION_politician                 MOD_homicide   
2  GENDER_male        OCCUPATION_artist  MOD_Unknown_Manner_of_Death   
3  GENDER_male        OCCUPATION_artist           MOD_natural causes   
4  GENDER_male        OCCUPATION_artist           MOD_natural causes   
5  GENDER_male    OCCUPATION_politician  MOD_Unknown_Manner_of_Death   
6  GENDER_male  OCCUPATION_statesperson                  MOD_suicide   
7  GENDER_male  OCCUPATION_statesperson                  MOD_suicide   
8  GENDER_male  OCCUPATION_statesperson                  MOD_suicide   
9  GENDER_male       OCCUPATION_athlete  MOD_Unknown_Manner_of_Death   

     AOD_Category    Associated Countries  
0   AOD_CAT_41-50  COUNTRY_united kingdom  
1   AOD_CAT_51-60   COUNTRY_united states  
2   AOD_CAT_81-90          COUNTRY_france  
3   AOD_CAT_81-90          COUNTRY_france  
4   AOD_CAT_41-50   COUNTRY_united states  
5  AOD_CAT_91-100           COUNTRY_chile  
6   AOD_CAT_51-60         COUNTRY_austria  
7   AOD_CAT_51-60         COUNTRY_germany  
8   AOD_CAT_51-60         COUNTRY_hungary  
9   AOD_CAT_61-70     COUNTRY_netherlands

Let's investigate any relationship between country and occupation. We will drop all other columns. We use a fairly low minimum support threshold of 0.01 as there are many possible combinations of country and occupation.

In [88]:
min_sup = 0.005
min_conf = 0.25

res_df = trans_df[['Associated Countries', 'Occupation']]
fp_tree = Tree(res_df.values.tolist(), min_sup=min_sup)
frequent_itemsets = fp_tree.generate_frequent_itemsets()
print(f"{len(frequent_itemsets)} frequent itemsets generated")
association_rules = fp_tree.generate_rules(frequent_itemsets, min_conf)
print(f"{len(association_rules)} rules generated")
for rule in association_rules:
    print(rule)

75 frequent itemsets generated
13 rules generated
['COUNTRY_australia'] -> ['OCCUPATION_athlete'] | Confidence: 30.36% | Lift: 2.51
['COUNTRY_hungary'] -> ['OCCUPATION_artist'] | Confidence: 31.97% | Lift: 1.19
['COUNTRY_netherlands'] -> ['OCCUPATION_artist'] | Confidence: 31.65% | Lift: 1.17
['COUNTRY_spain'] -> ['OCCUPATION_artist'] | Confidence: 34.96% | Lift: 1.30
['COUNTRY_austria'] -> ['OCCUPATION_artist'] | Confidence: 31.96% | Lift: 1.19
['COUNTRY_canada'] -> ['OCCUPATION_politician'] | Confidence: 42.15% | Lift: 2.39
['COUNTRY_russia'] -> ['OCCUPATION_artist'] | Confidence: 29.43% | Lift: 1.09
['COUNTRY_sweden'] -> ['OCCUPATION_artist'] | Confidence: 33.65% | Lift: 1.25
['COUNTRY_italy'] -> ['OCCUPATION_artist'] | Confidence: 29.58% | Lift: 1.10
['COUNTRY_france'] -> ['OCCUPATION_artist'] | Confidence: 29.69% | Lift: 1.10
['COUNTRY_united kingdom'] -> ['OCCUPATION_athlete'] | Confidence: 25.27% | Lift: 2.09
['OCCUPATION_athlete'] -> ['COUNTRY_united states'] | Confidence: 26.6

Many of the relationships include the occupation of artist. Let's look at the distribution of occupations to see if there is a skew in the data.

In [89]:
occupation_counts = trans_df['Occupation'].value_counts()

# calculate the percentage of each unique value
percentage = (occupation_counts / len(trans_df)) * 100

# create a df to display the results
occupation_distribution = pd.DataFrame({'Count': occupation_counts, 'Percentage': percentage})

# Display the gender distribution DataFrame
occupation_distribution.head(10)

Count  Percentage
Occupation                                       
OCCUPATION_artist              213252   26.961979
OCCUPATION_politician          139735   17.667043
OCCUPATION_athlete              95819   12.114634
OCCUPATION_researcher           64208    8.117977
OCCUPATION_Unknown_Occupation   60169    7.607316
OCCUPATION_military personnel   30609    3.869972
OCCUPATION_religious figure     13958    1.764745
OCCUPATION_businessperson       13411    1.695586
OCCUPATION_architect            12938    1.635783
OCCUPATION_journalist           12586    1.591279

Over 25% of entries are artists, and over 17% are politians. This explains the majority of the associated rules that were mined. Perhaps the only interesting rule that has been mined is that athletes are strongly associated with the United States and Australia.

Any relationship between gender and other attributes is most likely misleading. This is because a significant portion of the dataset are male (over 79%).

In [90]:
gender_counts = trans_df['Gender'].value_counts()

# calculate the percentage of each unique value
percentage = (gender_counts / len(trans_df)) * 100

# create a df to display the results
gender_distribution = pd.DataFrame({'Count': gender_counts, 'Percentage': percentage})

# Display the gender distribution DataFrame
gender_distribution

Count  Percentage
Gender                                       
GENDER_male                632006   79.906086
GENDER_Unknown_Gender       79406   10.039498
GENDER_female               79399   10.038612
GENDER_transgender female      76    0.009609
GENDER_transgender male        31    0.003919
GENDER_intersex                13    0.001644
GENDER_transgender person       2    0.000253
GENDER_non-binary               2    0.000253
GENDER_eunuch                   1    0.000126

Now let's look at the relationship between occupation and age of death

In [91]:
min_sup = 0.005
min_conf = 0.25

res_df = trans_df[['AOD_Category', 'Occupation']]
fp_tree = Tree(res_df.values.tolist(), min_sup=min_sup)
frequent_itemsets = fp_tree.generate_frequent_itemsets()
print(f"{len(frequent_itemsets)} frequent itemsets generated")
association_rules = fp_tree.generate_rules(frequent_itemsets, min_conf)
print(f"{len(association_rules)} rules generated")
for rule in association_rules:
    print(rule)

58 frequent itemsets generated
15 rules generated
['OCCUPATION_religious figure'] -> ['AOD_CAT_71-80'] | Confidence: 28.89% | Lift: 1.08
['OCCUPATION_religious figure'] -> ['AOD_CAT_81-90'] | Confidence: 28.94% | Lift: 1.20
['AOD_CAT_31-40'] -> ['OCCUPATION_artist'] | Confidence: 28.44% | Lift: 1.05
['AOD_CAT_41-50'] -> ['OCCUPATION_artist'] | Confidence: 30.41% | Lift: 1.13
['OCCUPATION_Unknown_Occupation'] -> ['AOD_CAT_71-80'] | Confidence: 25.19% | Lift: 0.94
['AOD_CAT_91-100'] -> ['OCCUPATION_artist'] | Confidence: 26.58% | Lift: 0.99
['OCCUPATION_researcher'] -> ['AOD_CAT_71-80'] | Confidence: 28.39% | Lift: 1.06
['OCCUPATION_researcher'] -> ['AOD_CAT_81-90'] | Confidence: 28.45% | Lift: 1.18
['AOD_CAT_51-60'] -> ['OCCUPATION_artist'] | Confidence: 29.21% | Lift: 1.08
['OCCUPATION_athlete'] -> ['AOD_CAT_71-80'] | Confidence: 26.27% | Lift: 0.98
['OCCUPATION_politician'] -> ['AOD_CAT_71-80'] | Confidence: 29.51% | Lift: 1.10
['AOD_CAT_61-70'] -> ['OCCUPATION_artist'] | Confidence: 

Let's investigate the distribution of the age of death throughout the database.

In [92]:
AOD_counts = trans_df['AOD_Category'].value_counts()

# calculate the percentage of each unique value
percentage = (AOD_counts / len(trans_df)) * 100

# create a df to display the results
AOD_distribution = pd.DataFrame({'Count': AOD_counts, 'Percentage': percentage})

# Display the gender distribution DataFrame
AOD_distribution

Count  Percentage
AOD_Category                      
AOD_CAT_71-80   212099   26.816203
AOD_CAT_81-90   191341   24.191717
AOD_CAT_61-70   151937   19.209772
AOD_CAT_51-60    84764   10.716923
AOD_CAT_91-100   63244    7.996096
AOD_CAT_41-50    43172    5.458343
AOD_CAT_31-40    25599    3.236545
AOD_CAT_21-30    15815    1.999530
AOD_CAT_11-20     2114    0.267278
AOD_CAT_100+       693    0.087618
AOD_CAT_0-10       158    0.019976

A significant portion of our dataset fall in the 71-80, 81-90, and 61-70 categories. Most of the rules include artists, which were known to resemble a large amount of the data points. It is interesting to note however that religious figures commonly have long lives.

Now let's look at the relationship between occupation and manner of death

In [93]:
min_sup = 0.005
min_conf = 0.25

res_df = trans_df[['Manner of death', 'Occupation']]
fp_tree = Tree(res_df.values.tolist(), min_sup=min_sup)
frequent_itemsets = fp_tree.generate_frequent_itemsets()
print(f"{len(frequent_itemsets)} frequent itemsets generated")
association_rules = fp_tree.generate_rules(frequent_itemsets, min_conf)
print(f"{len(association_rules)} rules generated")
for rule in association_rules:
    print(rule)

37 frequent itemsets generated
17 rules generated
['OCCUPATION_lawyer'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 94.79% | Lift: 1.01
['OCCUPATION_judge'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 96.51% | Lift: 1.03
['OCCUPATION_physician'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 96.74% | Lift: 1.03
['OCCUPATION_engineer'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 96.34% | Lift: 1.03
['OCCUPATION_teacher'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 96.53% | Lift: 1.03
['OCCUPATION_journalist'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 92.36% | Lift: 0.98
['OCCUPATION_architect'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 98.73% | Lift: 1.05
['OCCUPATION_businessperson'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 94.18% | Lift: 1.00
['OCCUPATION_religious figure'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 96.12% | Lift: 1.02
['OCCUPATION_military personnel'] -> ['MOD_Unknown_Manner_of_Death'] | Confidence: 93.89% | Lift: 1.0

Most of these rules produce an unknown manner of death. Let's see how many samples did not include a manner of death.

In [94]:
mod_counts = trans_df['Manner of death'].value_counts()

# calculate the percentage of each unique value
percentage = (mod_counts / len(trans_df)) * 100

# create a df to display the results
mod_distribution = pd.DataFrame({'Count': mod_counts, 'Percentage': percentage})

# Display the gender distribution DataFrame
mod_distribution

Count  Percentage
Manner of death                                              
MOD_Unknown_Manner_of_Death                742643   93.894196
MOD_natural causes                          32421    4.099067
MOD_suicide                                  4977    0.629254
MOD_accident                                 4756    0.601313
MOD_homicide                                 3713    0.469444
...                                           ...         ...
MOD_cardiac arrest                              1    0.000126
MOD_substance abuse                             1    0.000126
MOD_chronic obstructive pulmonary disease       1    0.000126
MOD_rape                                        1    0.000126
MOD_stomach cancer                              1    0.000126

[139 rows x 2 columns]

Over 90% of the data! It is a good idea to drop this information and attempt to find some new rules.

In [95]:
res_df = res_df[res_df['Manner of death'] != 'MOD_Unknown_Manner_of_Death']
fp_tree = Tree(res_df.values.tolist(), min_sup=min_sup)
frequent_itemsets = fp_tree.generate_frequent_itemsets()
print(f"{len(frequent_itemsets)} frequent itemsets generated")
association_rules = fp_tree.generate_rules(frequent_itemsets, min_conf)
print(f"{len(association_rules)} rules generated")
for rule in association_rules:
    print(rule)

44 frequent itemsets generated
14 rules generated
['OCCUPATION_teacher'] -> ['MOD_natural causes'] | Confidence: 67.99% | Lift: 1.01
['OCCUPATION_religious figure'] -> ['MOD_natural causes'] | Confidence: 64.33% | Lift: 0.96
['OCCUPATION_businessperson'] -> ['MOD_natural causes'] | Confidence: 70.04% | Lift: 1.04
['OCCUPATION_journalist'] -> ['MOD_natural causes'] | Confidence: 66.60% | Lift: 0.99
['OCCUPATION_Unknown_Occupation'] -> ['MOD_homicide'] | Confidence: 33.26% | Lift: 4.33
['OCCUPATION_military personnel'] -> ['MOD_natural causes'] | Confidence: 36.24% | Lift: 0.54
['OCCUPATION_researcher'] -> ['MOD_natural causes'] | Confidence: 73.91% | Lift: 1.10
['MOD_homicide'] -> ['OCCUPATION_politician'] | Confidence: 26.21% | Lift: 1.86
['MOD_accident'] -> ['OCCUPATION_artist'] | Confidence: 26.39% | Lift: 0.69
['MOD_suicide'] -> ['OCCUPATION_artist'] | Confidence: 38.88% | Lift: 1.01
['OCCUPATION_athlete'] -> ['MOD_natural causes'] | Confidence: 68.94% | Lift: 1.03
['OCCUPATION_poli

It seems there are no interesting rules here. Instead of unknown, any rules generated no include natural causes.

Now let's look at the relationship between manner of death and age of death, where the unknown manner of death entries have been dropped

In [96]:
min_sup = 0.005
min_conf = 0.25

res_df = trans_df[['AOD_Category', 'Manner of death']]
res_df = res_df[res_df['Manner of death'] != 'MOD_Unknown_Manner_of_Death']
fp_tree = Tree(res_df.values.tolist(), min_sup=min_sup)
frequent_itemsets = fp_tree.generate_frequent_itemsets()
print(f"{len(frequent_itemsets)} frequent itemsets generated")
association_rules = fp_tree.generate_rules(frequent_itemsets, min_conf)
print(f"{len(association_rules)} rules generated")
for rule in association_rules:
    print(rule)

44 frequent itemsets generated
9 rules generated
['AOD_CAT_91-100'] -> ['MOD_natural causes'] | Confidence: 94.05% | Lift: 1.40
['MOD_capital punishment'] -> ['AOD_CAT_41-50'] | Confidence: 26.25% | Lift: 1.98
['AOD_CAT_21-30'] -> ['MOD_accident'] | Confidence: 25.43% | Lift: 2.58
['AOD_CAT_31-40'] -> ['MOD_natural causes'] | Confidence: 32.68% | Lift: 0.49
['AOD_CAT_81-90'] -> ['MOD_natural causes'] | Confidence: 91.91% | Lift: 1.37
['AOD_CAT_41-50'] -> ['MOD_natural causes'] | Confidence: 46.09% | Lift: 0.69
['AOD_CAT_71-80'] -> ['MOD_natural causes'] | Confidence: 88.98% | Lift: 1.33
['AOD_CAT_51-60'] -> ['MOD_natural causes'] | Confidence: 65.58% | Lift: 0.98
['AOD_CAT_61-70'] -> ['MOD_natural causes'] | Confidence: 80.03% | Lift: 1.19


Again, mostly natural causes. However it is interesting that accidents are associated with an early age of death between 21 and 30.

Now let's look at the relationship between manner of death and country, where the unknown manner of death entries have been dropped

In [97]:
min_sup = 0.005
min_conf = 0.25

res_df = trans_df[['Manner of death', 'Associated Countries']]
res_df = res_df[res_df['Manner of death'] != 'MOD_Unknown_Manner_of_Death']
fp_tree = Tree(res_df.values.tolist(), min_sup=min_sup)
frequent_itemsets = fp_tree.generate_frequent_itemsets()
print(f"{len(frequent_itemsets)} frequent itemsets generated")
association_rules = fp_tree.generate_rules(frequent_itemsets, min_conf)
print(f"{len(association_rules)} rules generated")
for rule in association_rules:
    print(rule)

60 frequent itemsets generated
21 rules generated
['COUNTRY_argentina'] -> ['MOD_natural causes'] | Confidence: 73.19% | Lift: 1.09
['COUNTRY_brazil'] -> ['MOD_natural causes'] | Confidence: 72.58% | Lift: 1.08
['COUNTRY_mexico'] -> ['MOD_natural causes'] | Confidence: 58.17% | Lift: 0.87
['COUNTRY_netherlands'] -> ['MOD_natural causes'] | Confidence: 56.80% | Lift: 0.85
['COUNTRY_hungary'] -> ['MOD_natural causes'] | Confidence: 44.11% | Lift: 0.66
['COUNTRY_poland'] -> ['MOD_natural causes'] | Confidence: 38.12% | Lift: 0.57
['COUNTRY_austria'] -> ['MOD_natural causes'] | Confidence: 57.65% | Lift: 0.86
['COUNTRY_spain'] -> ['MOD_homicide'] | Confidence: 26.24% | Lift: 3.41
['COUNTRY_spain'] -> ['MOD_natural causes'] | Confidence: 49.38% | Lift: 0.74
['COUNTRY_australia'] -> ['MOD_natural causes'] | Confidence: 76.40% | Lift: 1.14
['COUNTRY_japan'] -> ['MOD_natural causes'] | Confidence: 54.32% | Lift: 0.81
['COUNTRY_canada'] -> ['MOD_natural causes'] | Confidence: 82.25% | Lift: 1.2

Now let's look at the relationship between age of death and country

In [98]:
min_sup = 0.005
min_conf = 0.25

res_df = trans_df[['AOD_Category', 'Associated Countries']]
fp_tree = Tree(res_df.values.tolist(), min_sup=min_sup)
frequent_itemsets = fp_tree.generate_frequent_itemsets()
print(f"{len(frequent_itemsets)} frequent itemsets generated")
association_rules = fp_tree.generate_rules(frequent_itemsets, min_conf)
print(f"{len(association_rules)} rules generated")
for rule in association_rules:
    print(rule)

89 frequent itemsets generated
20 rules generated
['COUNTRY_australia'] -> ['AOD_CAT_71-80'] | Confidence: 27.84% | Lift: 1.04
['COUNTRY_norway'] -> ['AOD_CAT_81-90'] | Confidence: 27.66% | Lift: 1.14
['COUNTRY_norway'] -> ['AOD_CAT_71-80'] | Confidence: 29.14% | Lift: 1.09
['COUNTRY_hungary'] -> ['AOD_CAT_71-80'] | Confidence: 26.62% | Lift: 0.99
['COUNTRY_japan'] -> ['AOD_CAT_71-80'] | Confidence: 25.87% | Lift: 0.96
['COUNTRY_japan'] -> ['AOD_CAT_81-90'] | Confidence: 27.26% | Lift: 1.13
['COUNTRY_netherlands'] -> ['AOD_CAT_71-80'] | Confidence: 27.48% | Lift: 1.02
['COUNTRY_spain'] -> ['AOD_CAT_71-80'] | Confidence: 25.31% | Lift: 0.94
['COUNTRY_austria'] -> ['AOD_CAT_71-80'] | Confidence: 27.06% | Lift: 1.01
['COUNTRY_canada'] -> ['AOD_CAT_81-90'] | Confidence: 25.51% | Lift: 1.05
['COUNTRY_canada'] -> ['AOD_CAT_71-80'] | Confidence: 27.48% | Lift: 1.02
['COUNTRY_russia'] -> ['AOD_CAT_71-80'] | Confidence: 25.87% | Lift: 0.96
['COUNTRY_sweden'] -> ['AOD_CAT_81-90'] | Confidence: 2

Let's drop the gender column before diving into association rules for each individual country, as it is meaningless.

In [99]:
trans_df = trans_df.drop('Gender', axis=1)
trans_df = trans_df[trans_df['Manner of death'] != 'MOD_Unknown_Manner_of_Death']
trans_df.head(10)

Occupation     Manner of death    AOD_Category   
0         OCCUPATION_artist  MOD_natural causes   AOD_CAT_41-50  \
1     OCCUPATION_politician        MOD_homicide   AOD_CAT_51-60   
3         OCCUPATION_artist  MOD_natural causes   AOD_CAT_81-90   
4         OCCUPATION_artist  MOD_natural causes   AOD_CAT_41-50   
6   OCCUPATION_statesperson         MOD_suicide   AOD_CAT_51-60   
7   OCCUPATION_statesperson         MOD_suicide   AOD_CAT_51-60   
8   OCCUPATION_statesperson         MOD_suicide   AOD_CAT_51-60   
10    OCCUPATION_politician  MOD_natural causes  AOD_CAT_91-100   
15        OCCUPATION_artist        MOD_accident   AOD_CAT_51-60   
16        OCCUPATION_artist  MOD_natural causes   AOD_CAT_31-40   

      Associated Countries  
0   COUNTRY_united kingdom  
1    COUNTRY_united states  
3           COUNTRY_france  
4    COUNTRY_united states  
6          COUNTRY_austria  
7          COUNTRY_germany  
8          COUNTRY_hungary  
10           COUNTRY_chile  
15   COUNTRY_united states  
16         COUNTRY_jamaica

Now let's investigate all generated rules for each individual country. We will only consider those countries which over over 1000 entries. Those with less may produce interesting rules at the expense of real life accuracy.

In [109]:
min_sup = 0.1
min_conf = 0.5

for country in trans_df['Associated Countries'].unique():
    country_df = trans_df[trans_df['Associated Countries'] == country]
    country_df = country_df.drop('Associated Countries', axis=1)
    if country_df.shape[0] > 1000:
        print(f'Calculating association rules for the country {country[8:]} with {country_df.shape[0]} entries')
        
        fp_tree = Tree(country_df.values.tolist(), min_sup=min_sup)
        frequent_itemsets = fp_tree.generate_frequent_itemsets()
        print(f"{len(frequent_itemsets)} frequent itemsets generated")
        association_rules = fp_tree.generate_rules(frequent_itemsets, min_conf)
        print(f"{len(association_rules)} rules generated")
        for rule in association_rules:
            print(rule)
        print('',end='\n\n')

Calculating association rules for the country united kingdom with 4953 entries
15 frequent itemsets generated
6 rules generated
['AOD_CAT_81-90'] -> ['MOD_natural causes'] | Confidence: 92.42% | Lift: 1.20
['OCCUPATION_athlete'] -> ['MOD_natural causes'] | Confidence: 77.85% | Lift: 1.01
['AOD_CAT_71-80'] -> ['MOD_natural causes'] | Confidence: 92.94% | Lift: 1.21
['AOD_CAT_51-60'] -> ['MOD_natural causes'] | Confidence: 76.86% | Lift: 1.00
['AOD_CAT_61-70'] -> ['MOD_natural causes'] | Confidence: 90.81% | Lift: 1.18
['OCCUPATION_artist'] -> ['MOD_natural causes'] | Confidence: 84.87% | Lift: 1.10


Calculating association rules for the country united states with 18572 entries
14 frequent itemsets generated
5 rules generated
['AOD_CAT_81-90'] -> ['MOD_natural causes'] | Confidence: 93.99% | Lift: 1.21
['AOD_CAT_51-60'] -> ['MOD_natural causes'] | Confidence: 76.50% | Lift: 0.98
['AOD_CAT_71-80'] -> ['MOD_natural causes'] | Confidence: 92.93% | Lift: 1.20
['AOD_CAT_61-70'] -> ['MOD_natu

There aren't many interesting rules here that have not been previously exposed. Most rules include the natural causes manner of death. This is not indicative of any meaningful 'way of life', and hence does not provide any value.